<a href="https://colab.research.google.com/github/mr-cri-spy/SLM/blob/main/FAISS_Based_Chatbot_with_Confidence_and_Fallback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q faiss-cpu sentencepiece transformers torch
!pip install -q git+https://github.com/ai-forever/ru-transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/ai-forever/ru-transform

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

Load Multilingual Embedding Model (e.g., IndicBERT)

In [3]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [4]:
def get_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

Sample multilingual knowledge base

In [5]:
faq_data = [
    {"question": "Where are you located?", "answer": "We are based in Mysore city, Karnataka, India."},
    {"question": "What services do you offer?", "answer": "We provide AI chatbot and automation services."},
    {"question": "How to contact you?", "answer": "You can reach us via email or phone listed on our website."},
    {"question": "ನೀವು ಎಲ್ಲಿ ಇದ್ದೀರಿ?", "answer": "ನಾವು ಕರ್ನಾಟಕ ರಾಜ್ಯದ ಮೈಸೂರಿನಲ್ಲಿ ಇದ್ದೇವೆ."},
    {"question": "तुम्ही कुठे आहात?", "answer": "आम्ही भारतातील कर्नाटकमधील मैसूर येथे आहोत."}
]

Create embeddings and FAISS index

In [6]:
dimension = get_embedding("test").shape[0]
index = faiss.IndexFlatL2(dimension)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
faq_embeddings = []
for item in faq_data:
    emb = get_embedding(item["question"])
    faq_embeddings.append(emb)
    index.add(np.array([emb]))


Answer function with confidence scoring and fallback

In [8]:
def ask_bot(user_query, k=3, threshold=0.4):
    query_vector = get_embedding(user_query).reshape(1, -1)
    distances, indices = index.search(query_vector, k)

    best_idx = indices[0][0]
    best_distance = distances[0][0]
    confidence = 1 / (1 + best_distance)

    if confidence < threshold:
        return f" I'm not sure I understood that. Please try again. (conf: {confidence:.2f})"

    response = faq_data[best_idx]['answer']
    return f" Answer: {response}\n(confidence: {confidence:.2f})"

Test

In [9]:
print(ask_bot("ur company location"))
print(ask_bot("आपकी सेवा क्या है?"))
print(ask_bot("tell me something unknown"))

 I'm not sure I understood that. Please try again. (conf: 0.04)
 I'm not sure I understood that. Please try again. (conf: 0.06)
 I'm not sure I understood that. Please try again. (conf: 0.05)


We can Use T5 Paraphraser

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Loading T5-small paraphraser
t5_tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5_paraphraser")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5_paraphraser")

def paraphrase(text):
    input_text = f"paraphrase: {text} </s>"
    encoding = t5_tokenizer.encode_plus(input_text, return_tensors="pt", padding=True)
    output = t5_model.generate(
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"],
        max_length=64,
        num_beams=4,
        num_return_sequences=1,
        temperature=1.5
    )
    return t5_tokenizer.decode(output[0], skip_special_tokens=True)




In [13]:

response = "We are based in Mysore city, Karnataka."
print(paraphrase(response))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


We are based in Mysore, Karnataka.
